In [58]:
import numpy as np
import pandas as pd
import sys
import os
import warnings
import chart_studio
import chart_studio.plotly as cspy
import chart_studio.tools as tls

from sample_data import network
from tesla_supercharger_web_parser import tesla_supercharger_parser
from print_utils import Printer
from utils import read_csv, add_random_charging_rates, plot_superchargers, plot_superchargers_with_path
from planner import planner, backtrack, display_path
from node import Node
from credentials import username, api_key
from global_vars import min_charging_rate, max_charging_rate ,decimal_places ,verbose ,velocity ,max_dist ,earth_radius, parse, webscraper_data, data_directory

if not os.path.exists(data_directory):
	    # If it doesn't exist, create it
	    os.makedirs(data_directory)

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
os.system('cls' if os.name == 'nt' else 'clear')

# generate web-scraped dataset into a csv
if parse:
	tesla_supercharger_parser()

# use web-scraped dataset
if webscraper_data:
	data = read_csv(data_directory+"supercharger-1.csv")
	data.columns = ["Location","lat","lng"]
	data = add_random_charging_rates(data,min_charging_rate,max_charging_rate)
	df = data.sample(n=2)
	start_idx = df.index.tolist()[0]
	goal_idx = df.index.tolist()[1]
	start = data.iloc[start_idx]
	goal = data.iloc[goal_idx]

	start_node = Node(start.Location,start.lat,start.lng,start.charging_rate)
	goal_node = Node(goal.Location,goal.lat,goal.lng,goal.charging_rate)

# use sample network from coding challenge
else:
	network_df = pd.DataFrame(network,columns=["Location","lat","lng","charging_rate"])
	network_df.to_csv(data_directory+"network.csv")
	data = network_df
	start = data[data.Location=="Council_Bluffs_IA"]
	goal = data[data.Location=="Cadillac_MI"]
	start_idx = start.index[0]
	goal_idx = goal.index[0]
	start_node = Node(start.Location.values.tolist()[0],start.lat.values.tolist()[0],start.lng.values.tolist()[0],start.charging_rate.values.tolist()[0])
	goal_node = Node(goal.Location.values.tolist()[0],goal.lat.values.tolist()[0],goal.lng.values.tolist()[0],goal.charging_rate.values.tolist()[0])

# remove start_node
query = data.copy()
query = query.drop([start_idx])

# create Node classes for each charging station
nodes = []
for i in range(len(query)):
	row = query.iloc[i]
	sample = Node(row.Location,row.lat,row.lng,row.charging_rate)
	nodes.append(sample)

# set the inital fuel charge to be the max distance it can travel
start_node.set_remaining_fuel_distance(max_dist)

# a-star planner with heuristics
status = planner(start_node,goal_node,nodes)
if isinstance(status, Node):
    # generate backtracked path
	path = backtrack(status)
	# plot backtracked path using plotly
	pathdf = display_path(data,path)
	# fig = plot_superchargers_with_path(data,pathdf)
else:
    # no path
	path = None
	warnings.warn("Planner couldn't find a path. Terminating...")
	sys.exit()

 ('Council_Bluffs_IA', (41.220921, -95.835579), 165.0, 8.66163305034881, 855.0586472780235)
 ('Worthington_MN', (43.63385, -95.595647), 108.0, 10.786416085607863, 787.6863374950125, 'Council_Bluffs_IA', 268.30673534846466)
 ('Albert_Lea_MN', (43.68606, -93.357721), 92.0, 8.17160068091828, 616.1869227355636, 'Worthington_MN', 171.50546736168818)
 ('Mauston_WI', (43.795551, -90.059358), 138.0, 6.133431627717298, 363.09954879487435, 'Albert_Lea_MN', 253.28363253322328)
 ('Sheboygan_WI', (43.749753, -87.746971), 116.0, 3.6654030002225624, 190.1111642282576, 'Mauston_WI', 177.54781265376056)
 ('Cadillac_MI', (44.28254, -85.40306), 111.0, 1.8105825164595961, 0.0, 'Sheboygan_WI', 190.1111642282576)
 Goal reached!
 Path Results:
            Location        lat        lng  charging_rate  dist_from_parent  \
0  Council_Bluffs_IA  41.220921 -95.835579          165.0          0.000000   
1     Worthington_MN  43.633850 -95.595647          108.0        268.306735   
2      Albert_Lea_MN  43.686060 

In [74]:
import plotly
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from global_vars import min_charging_rate, max_charging_rate ,decimal_places ,verbose ,velocity ,max_dist ,earth_radius, parse
from credentials import mapbox_token
import chart_studio
import chart_studio.plotly as cspy
import chart_studio.tools as tls

data['val'] = data.Location.astype(str)+": "+data.charging_rate.astype(str)

tmp = pathdf.select_dtypes(include=[np.float64])
pathdf.loc[:, tmp.columns] = np.round(tmp,2)

pathdf['val'] = "Distance to reach "+pathdf.Location.astype(str)+": "+pathdf.dist_from_parent.astype(str)+"\nCharging Rate of station:"+pathdf.charging_rate.astype(str)+"\nCharging Time:"+pathdf.time_spent_charging.astype(str)+"\nDriving Time:"+pathdf.time_spent_driving.astype(str)+"\nTotal Time spent to reach station and charge:"+pathdf.time_spent.astype(str)+"\nTotal Time spent cumulative:"+pathdf.total_time_spent.astype(str)


charging_stations  = go.Scattergeo(
    # hoverinfo='charging_rate',
    lon = data.lng,
    lat = data.lat,
    text = data.Location,
    mode = 'markers',
    marker = dict(
    	size = 5,
    	opacity = 0.75,
    	reversescale = False,
    	autocolorscale = False,
    	cmin = data.charging_rate.min(),
    	color = data.charging_rate,
    	cmax = data.charging_rate.max(),
    	colorbar_title="Charging Rate")
    )

fig = go.Figure(data=charging_stations)

fig.update_layout(
        title = 'Tesla Supercharger Network',
        geo_scope='usa',
        showlegend=False,
        mapbox = dict(
        accesstoken = mapbox_token,
        bearing=0,
        

),
    )

fig.show()
plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')


'<div>                            <div id="fddcb4ce-9612-4daf-877e-7f1d77a14e05" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("fddcb4ce-9612-4daf-877e-7f1d77a14e05")) {                    Plotly.newPlot(                        "fddcb4ce-9612-4daf-877e-7f1d77a14e05",                        [{"lat":[42.710356,40.544595,39.858702,43.592251,34.667629,35.108486,39.662265,43.623536,40.738399,38.877342,37.66976,38.249149,42.518715,39.040814,33.894227,32.726686,41.245823,43.102424,38.957778,40.277134,28.514873,30.181405,37.394181,45.70007,41.348503,45.64655,42.241125,34.850835,34.98737,33.79382,40.958869,40.66179,41.585206,35.174151,36.860525,29.112378,41.045431,46.729872,40.891909,36.61697,37.703163,38.017955,38.235578,44.347885,39.678111,38.993577,44.704537,43.109066,32.374263,40.32244,35.3407

In [75]:
chosen_charging_stations  = go.Scattergeo(
    # hoverinfo='charging_rate',
    lon = pathdf.lng,
    lat = pathdf.lat,
    text = pathdf.Location,
    mode = 'markers+lines',
    marker = dict(
    	size = 5,
    	opacity = 0.75,
    	reversescale = False,
    	autocolorscale = False,
    	cmin = data.charging_rate.min(),
    	color = pathdf.charging_rate,
    	cmax = data.charging_rate.max(),
    	colorbar_title="Charging Rate")
    )

fig = go.Figure(data=chosen_charging_stations)

fig.update_layout(
        title = 'Tesla Supercharger Network',
        geo_scope='usa',
        showlegend=False,
        mapbox = dict(
        accesstoken = mapbox_token,
        bearing=0,
        

),
    )

fig.show()
plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

'<div>                            <div id="b0b4c0d3-501b-4b43-b009-a836e189a63a" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("b0b4c0d3-501b-4b43-b009-a836e189a63a")) {                    Plotly.newPlot(                        "b0b4c0d3-501b-4b43-b009-a836e189a63a",                        [{"lat":[41.22,43.63,43.69,43.8,43.75,44.28],"lon":[-95.84,-95.6,-93.36,-90.06,-87.75,-85.4],"marker":{"autocolorscale":false,"cmax":194.0,"cmin":79.0,"color":[165.0,108.0,92.0,138.0,116.0,111.0],"colorbar":{"title":{"text":"Charging Rate"}},"opacity":0.75,"reversescale":false,"size":5},"mode":"markers+lines","text":["Council_Bluffs_IA","Worthington_MN","Albert_Lea_MN","Mauston_WI","Sheboygan_WI","Cadillac_MI"],"type":"scattergeo"}],                        {"geo":{"scope":"usa"},"mapbox":{"accesstoken":

In [76]:

charge_cond = pathdf.time_spent_charging > 0
charge_df = pathdf[charge_cond]

cond = data['Location'].isin(pathdf['Location'])
data.drop(data[cond].index, inplace = True)

plots = [
    
    go.Scattermapbox(
    # hoverinfo='none',
    text = pathdf.Location,
    lon = data.lng,
    lat = data.lat,
    mode='markers',
    marker = {'size': 5,
                'symbol': ["fuel"]*len(data),
                'allowoverlap': True,
            #   'color': pathdf.charging_rate,
                'cmin': data.charging_rate.min(),
                'cmax': data.charging_rate.max()
            }),

    go.Scattermapbox(
    text = pathdf.Location,
    lon = pathdf.lng,
    lat = pathdf.lat,
    mode='markers+lines',
    marker = {'size': 8,
                'symbol': ["fuel"]*len(pathdf),
                'allowoverlap': True,
            #   'color': pathdf.charging_rate,
            }
    ),


        ]

fig = go.Figure(data=plots)

fig.update_layout(
        title = 'Tesla Supercharger Optimized Path from '+pathdf.Location.values.tolist()[0].replace("_"," ")+" to "+pathdf.Location.values.tolist()[-1].replace("_"," "),
        geo_scope='usa',
        showlegend=False,
        mapbox = dict(
        accesstoken = mapbox_token,
        bearing=0,
        center=dict(
            lat=38.92,
            lon=-99.3
        ),
        pitch=0,
        zoom=2.5,

),
    )

fig.update_traces(name='Station', showlegend = False)
fig.show()
plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

'<div>                            <div id="6bb53630-236b-4f1c-84af-db275c8048ea" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("6bb53630-236b-4f1c-84af-db275c8048ea")) {                    Plotly.newPlot(                        "6bb53630-236b-4f1c-84af-db275c8048ea",                        [{"lat":[42.710356,40.544595,39.858702,43.592251,34.667629,35.108486,39.662265,43.623536,40.738399,38.877342,37.66976,38.249149,42.518715,39.040814,33.894227,32.726686,41.245823,43.102424,38.957778,40.277134,28.514873,30.181405,37.394181,45.70007,41.348503,45.64655,42.241125,34.850835,34.98737,33.79382,40.958869,40.66179,41.585206,35.174151,36.860525,29.112378,41.045431,46.729872,40.891909,36.61697,37.703163,38.017955,38.235578,44.347885,39.678111,38.993577,44.704537,43.109066,32.374263,40.32244,35.3407